In [2]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine
from langchain.chains import create_sql_query_chain
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain_community.utilities import SQLDatabase
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import os
import warnings
from langchain_core.runnables import RunnablePassthrough


In [3]:
import pyodbc

In [4]:
load_dotenv(os.path.join(os.getcwd(),"Ambiente.env"))
azure_openai_endpoint = os.getenv("openai_endpoint")
azure_openai_key = os.getenv("openai_key")

In [5]:
llm_risposta1 = AzureChatOpenAI(
        deployment_name ="init-test-gpt-35-turbo",
        model="gpt-35-turbo",
        azure_endpoint = azure_openai_endpoint,
        openai_api_type="azure",
        openai_api_version = '2023-05-15',
        openai_api_key = azure_openai_key,
        temperature=0)


answer_prompt_risposta1 = PromptTemplate.from_template(
    """
    Data una frase in input {question} interpreta la frase in modo tale da identificare le seguenti possibilità:

    -Restituisci 1 se nella frase è presente la parola oggi
    -Restituisci 2 se nella domanda è presente la parola domani
    -Restituisci 3 in tutti gli altri casi.

    Restituisci solo il numero nella risposta.

    Question: {question}
    Risposta:
    """
)
chain_risposta = answer_prompt_risposta1 | llm_risposta1 

#creiamo di nuovo la catena che descriverà il modo in cui verrà elaborata la Query e la risposta


In [8]:
risposta= chain_risposta.invoke({"question": "Quali sono le mie prenotazioni di domani"}).content
risposta

'2'

In [9]:
print(risposta)

2


In [6]:
def mostra_prenotazioni(numero,id_cliente):
    con= pyodbc.connect('DRIVER={SQL Server};SERVER=bubidb.database.windows.net;DATABASE=mlacademy-sqldb;UID=MLacademy;PWD=alten-ML-academy2023')
    cursor= con.cursor()
    
    if numero==1: #oggi
        insert_query = """ SELECT luogo, orario, giorno, numero_persone
                           FROM Prenotazioni
                           WHERE id_Cliente=? AND Giorno = CONVERT(date, GETDATE());"""
    elif numero==2: #domani
        insert_query="""SELECT luogo, orario, giorno, numero_persone
                        FROM Prenotazioni
                        WHERE id_Cliente=? AND Giorno = CONVERT(date, GETDATE()+1);"""
    
    else:
        insert_query="""SELECT luogo, orario, giorno, numero_persone
                        FROM Prenotazioni
                        WHERE id_Cliente=? AND Giorno >= CONVERT(date, GETDATE());"""
    cursor.execute(insert_query,(id_cliente))
    
    lista_prenotazioni=[]
    for row in cursor.fetchall():
        lista_prenotazioni.append(tuple(row))

    cursor.close()
    con.commit()
    con.close()

    return lista_prenotazioni


In [10]:
lista=mostra_prenotazioni(int(risposta),1984)

In [11]:
import pandas as pd

In [129]:
lista

[('Milano', '12:00', '2023-07-25', 2),
 ('Roma', '14:00', '2023-07-26', 4),
 ('Napoli', '16:00', '2023-07-27', 3),
 ('Torino', '18:00', '2023-07-28', 1)]

In [135]:
for tupla in lista:
    print(tupla)

('Pizza360', '19:30-21:00', '2024-07-26', 4)
('Tao', '21:00-22:30', '2024-07-26', 4)
('My kimchi', '19:30-21:00', '2024-07-26', 4)
('Pizza360', '21:00-22:30', '2024-07-26', 6)


In [142]:
lista_tuples = [tuple(row) for row in lista]

In [12]:
df = pd.DataFrame(lista, columns=['luogo', 'orario', 'giorno', 'numero_persone'])

In [13]:
df

,luogo,orario,giorno,numero_persone
0,My kimchi,19:30-21:00,2024-07-27,1


In [26]:
for i in list(df.loc[0]):
    print(type(str(i)))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [34]:
lista_prenotazioni=[]
num_rows = df.shape[0]
for indice in range(num_rows):
    temporaneo=""
    for column in list(df.loc[indice]):
        temporaneo= temporaneo+" "+str(column)
    lista_prenotazioni.append(temporaneo.strip())

In [35]:
lista_prenotazioni

['My kimchi 19:30-21:00 2024-07-27 1']

In [127]:
lista = [
    ('Milano', '12:00', '2023-07-25', 2),
    ('Roma', '14:00', '2023-07-26', 4),
    ('Napoli', '16:00', '2023-07-27', 3),
    ('Torino', '18:00', '2023-07-28', 1)
]

# Creare il DataFrame dalla lista di tuple
df = pd.DataFrame(lista, columns=['luogo', 'orario', 'giorno', 'numero_persone'])

print(df)

    luogo orario      giorno  numero_persone
0  Milano  12:00  2023-07-25               2
1    Roma  14:00  2023-07-26               4
2  Napoli  16:00  2023-07-27               3
3  Torino  18:00  2023-07-28               1


In [ ]:
def elimina_prenotazione(prenotazione_da_eliminare,id_cliente):
    con= pyodbc.connect('DRIVER={SQL Server};SERVER=bubidb.database.windows.net;DATABASE=mlacademy-sqldb;UID=MLacademy;PWD=alten-ML-academy2023')
    cursor= con.cursor()

    delete_query = """ SELECT luogo, orario, giorno, numero_persone
                           FROM Prenotazioni
                           WHERE id_Cliente=? AND Giorno = CONVERT(date, GETDATE());"""
   
    cursor.execute(delete_query,(id_cliente))

    cursor.close()
    con.commit()
    con.close()
